In [2]:
from tqdm import tqdm
from workflow.fastani.remove_gunc_failed_contigs_by_contamination_sp_cluster import RemoveGuncFailedContigsByContaminationSpCluster
from workflow.config import PCT_VALUES
from workflow.external.gtdb_metadata import GtdbMetadataR207

In [3]:
DF = RemoveGuncFailedContigsByContaminationSpCluster().output().read_cached()
DF = DF[DF.index.get_level_values(1).isin(PCT_VALUES)]
print(DF.shape)

UNQ_GIDS = set(DF.index.get_level_values(0))

print(f'{len(UNQ_GIDS):,} failed gids')
DF.head()

(274977, 5)
35,714 failed gids


new_sp_rep      ani        af    type  same
gid             pct                                                  
GCA_000143435.1 1    GCF_001435955.1  97.3270  0.875591  sp_rep  True
                5    GCF_001435955.1  97.3270  0.875591  sp_rep  True
                15   GCF_001435955.1  97.6349  0.829228  sp_rep  True
GCA_000153745.1 5                NaN      NaN       NaN  no_ani  True
GCA_000155005.1 1    GCF_003697165.2  96.5203  0.827668  sp_rep  True

In [4]:
DF_META = GtdbMetadataR207().output().read_cached()
DF_META.head()

,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,contig_count,...,gtdb_type_species_of_genus,ncbi_taxonomy,ncbi_taxonomy_unfiltered,domain,phylum,class,order,family,genus,species
gid,,,,,,,,,,,,,,,,,,,,,
GCA_000006155.2,1916,93.12,0.0,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,426,...,f,d__Bacteria;p__Firmicutes;c__Bacilli;o__Bacill...,d__Bacteria;x__Terrabacteria group;p__Firmicut...,d__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae_G,g__Bacillus_A,s__Bacillus_A anthracis
GCA_000007325.1,1,99.95,0.0,149,k__Bacteria (UID2329),89,0.0,1973459,90.754610,1,...,t,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria;p__Fusobacteria;c__Fusobacteriia;o...,d__Bacteria,p__Fusobacteriota,c__Fusobacteriia,o__Fusobacteriales,f__Fusobacteriaceae,g__Fusobacterium,s__Fusobacterium nucleatum
GCA_000007385.1,0,99.82,0.0,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,1,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Xanthomonadales,f__Xanthomonadaceae,g__Xanthomonas,s__Xanthomonas oryzae
GCA_000008085.1,0,73.13,0.0,149,k__Archaea (UID2),107,0.0,462902,94.299480,1,...,t,d__Archaea;p__Nanoarchaeota;c__;o__Nanoarchaea...,d__Archaea;x__DPANN group;p__Nanoarchaeota;o__...,d__Archaea,p__Nanoarchaeota,c__Nanoarchaeia,o__Nanoarchaeales,f__Nanoarchaeaceae,g__Nanoarchaeum,s__Nanoarchaeum equitans
GCA_000008885.1,0,100.00,0.0,134,k__Bacteria (UID2495),80,0.0,617456,87.831079,2,...,f,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales_A,f__Enterobacteriaceae_A,g__Wigglesworthia,s__Wigglesworthia glossinidia_A


types:
no_af = no alignment fraction <0.5
no_ani = novel species cluster
sp_rep = found species representative within ANI radius


In [7]:

from collections import defaultdict, Counter


def get_ranks_for_cutoff_failed():

    df_changed = DF[DF['same'] == False]

    # print(df_changed.shape)
    # print(df_changed.head())


    d_pct_to_rank = defaultdict(list)
    for (gid, pct), row in df_changed.iterrows():
        d_pct_to_rank[pct].append(DF_META.loc[gid, 'genus'])

    # print(d_pct_to_rank)
    # print(cutoff_vals)
    # print(gids)
    # print(types)
    # print(new_sp_rep)

    for pct, ranks in d_pct_to_rank.items():
        vals = Counter(ranks)
        for rank, count in sorted(vals.items(), key=lambda x: x[1]):
            print(f'{pct:.2f} {rank} {count:,}')


    return d_pct_to_rank

get_ranks_for_cutoff_failed()


1.00 g__Acinetobacter 1
1.00 g__Anaerococcus 1
1.00 g__Veillonella 1
1.00 g__Klebsiella 1
1.00 g__Nosocomiicoccus 1
1.00 g__Geobacillus 1
1.00 g__Dietzia 1
1.00 g__Vibrio 1
1.00 g__Shewanella 1
1.00 g__Haemophilus 1
1.00 g__Xanthomonas 1
1.00 g__Proteus 1
1.00 g__Streptomyces 1
1.00 g__Collinsella 2
1.00 g__Pseudomonas_A 4
5.00 g__Acinetobacter 1
5.00 g__UBA822 1
5.00 g__UBA11600 1
5.00 g__Acidianus 1
5.00 g__MED-G11 1
5.00 g__Anaerococcus 1
5.00 g__Blautia_A 1
5.00 g__Veillonella 1
5.00 g__Klebsiella 1
5.00 g__Faecalicoccus 1
5.00 g__Lachnospira 1
5.00 g__Nosocomiicoccus 1
5.00 g__Brevundimonas 1
5.00 g__Geobacillus 1
5.00 g__Shewanella 1
5.00 g__Campylobacter_A 1
5.00 g__Haemophilus 1
5.00 g__Pseudomonas_E 1
5.00 g__Proteus 1
5.00 g__Streptomyces 1
5.00 g__Borreliella 2
5.00 g__Pseudomonas_A 4
5.00 g__Collinsella 4
10.00 g__Acinetobacter 1
10.00 g__UBA11600 1
10.00 g__Acidianus 1
10.00 g__Avimonas 1
10.00 g__Macondimonas 1
10.00 g__UBA1426 1
10.00 g__Anaerovibrio 1
10.00 g__MED-G11 1

defaultdict(list,
            {1: ['g__Acinetobacter',
              'g__Pseudomonas_A',
              'g__Anaerococcus',
              'g__Collinsella',
              'g__Collinsella',
              'g__Veillonella',
              'g__Klebsiella',
              'g__Nosocomiicoccus',
              'g__Pseudomonas_A',
              'g__Geobacillus',
              'g__Dietzia',
              'g__Vibrio',
              'g__Shewanella',
              'g__Haemophilus',
              'g__Xanthomonas',
              'g__Pseudomonas_A',
              'g__Proteus',
              'g__Pseudomonas_A',
              'g__Streptomyces'],
             5: ['g__Acinetobacter',
              'g__UBA822',
              'g__UBA11600',
              'g__Acidianus',
              'g__Pseudomonas_A',
              'g__MED-G11',
              'g__Anaerococcus',
              'g__Collinsella',
              'g__Collinsella',
              'g__Blautia_A',
              'g__Veillonella',
              'g__Klebsie